<a href="https://colab.research.google.com/github/Sharath-a26/Adversarial-Learning/blob/main/ASR_Calculation%20for%20FGSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
with ZipFile("drive/MyDrive/lc25000.zip") as zObject:
  zObject.extractall("dataset")

In [ ]:
import os
os.listdir("dataset/lung_colon_image_set/colon_image_sets")

['colon_n', 'colon_aca']

In [ ]:
import os
colon_folder = "dataset/lung_colon_image_set/colon_image_sets/colon_n"
for i in os.listdir(colon_folder):
  os.remove(colon_folder + "/" + i)
os.rmdir(colon_folder)

In [ ]:
lung_folder = "dataset/lung_colon_image_set/lung_image_sets/lung_n"
for i in os.listdir(lung_folder):
  os.remove(lung_folder + "/" + i)
os.rmdir(lung_folder)

In [ ]:
len(os.listdir("dataset/lung_colon_image_set/colon_image_sets/colon_aca"))

5000

In [ ]:
# Trying out for just 1 class

import tensorflow.keras as keras
dataset = keras.utils.image_dataset_from_directory(
    "dataset/lung_colon_image_set/colon_image_sets/colon_aca",image_size = (64,64),batch_size=32, label_mode=None
)

Found 5000 files.


In [ ]:
dataset = dataset.map(lambda x: x / 255.0)

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import torchvision

In [ ]:
class ResNetModel(nn.Module):
    def __init__(self, num_classes=5, pretrained = True):
        super(ResNetModel, self).__init__()

        if pretrained:
            weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2
            self.backbone = torchvision.models.resnet50(weights=weights)

        else:
            self.backbone = torchvision.models.resnet50(weights=None)

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.dropout(x)
        x = self.backbone.fc(x)

        return x

In [ ]:
resnet_model = ResNetModel(num_classes = 5, pretrained = False)

In [ ]:
resnet_model.load_state_dict(torch.load("drive/MyDrive/ResNet50_model.pth", map_location="cpu"))

<All keys matched successfully>

In [ ]:
resnet_model.eval()

ResNetModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [ ]:
from torchvision import transforms
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         ])

## Loading the FGSM model

In [ ]:
from tensorflow.keras.models import load_model

gen_model = load_model('/content/drive/My Drive/colon_aca_gen_with_fgsm_epoch_100.h5')

## Generating 1000 images from GAN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
input_latent_dim = 128

pred_images = []
for i in range(2000):
  random_noise = tf.random.normal(shape = (5,input_latent_dim))
  generated_images = gen_model(random_noise, training=False)

  generated_images = generated_images.numpy()


  img = generated_images[4]

  img = (img*255).astype(np.uint8)

  pred_images.append(img)

len(pred_images)
  # print(np.shape(img))
  # plt.imshow(img)

2000

In [ ]:
# transformming all 1000 images
from PIL import Image

trans_images = []
for i in range(len(pred_images)):
  img = Image.fromarray(pred_images[i])

  transformed_img = val_transform(img)
  trans_images.append(transformed_img.unsqueeze(0))

len(trans_images)



2000

In [ ]:


img1 = Image.open("dataset/lung_colon_image_set/colon_image_sets/colon_aca/colonca2472.jpeg").convert("RGB")

transformed_img = val_transform(img1)

AttributeError: 'Image' object has no attribute 'shape'

In [ ]:
transformed_img = transformed_img.unsqueeze(0)

## Sample Prediction with ResNet classifier

In [ ]:
all_preds = []
for i in range(len(trans_images)):
  with torch.no_grad():
    output = resnet_model(trans_images[i])
    probabilities = torch.softmax(output, dim=1)
    pred_class = torch.argmax(probabilities, dim=1).item()

    all_preds.append(pred_class)

    confidence = probabilities[0][pred_class].item()


In [ ]:
all_preds.count(0)

1776